# The River Pollution Problem

In [1]:
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.EAs.IMOEAD import IMOEA_D
from desdeo_problem import test_problem_builder
import plotly.express as px
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans

problem = test_problem_builder(name="DTLZ5", n_of_variables=12, n_of_objectives=3)


In [2]:
def PCP(evolver_rvea, evolver_moead, selected_kmeans = []):
    fig_rvea = px.parallel_coordinates(evolver_rvea.population.objectives)
    
    if (len(selected_kmeans) == 0):
        fig_moead = px.parallel_coordinates(evolver_moead.population.objectives)
        print(evolver_moead.population.objectives)
    else:
        fig_moead = px.parallel_coordinates(evolver_moead.population.objectives[selected_kmeans,:])
        print(evolver_moead.population.objectives[selected_kmeans,:])

    fig_moead.show()
    fig_rvea.show()


def plot_RVs(evolver):
    x = evolver.reference_vectors.values_planar[:,0]
    y = evolver.reference_vectors.values_planar[:,1]
    z = evolver.reference_vectors.values_planar[:,2]
    trace = go.Scatter3d(x = x, y = y, z= z, mode = "markers", marker_size=2)
    fig = go.Figure(data= [trace])
    fig.show()

def cluster_select(evolver, num_clusters):
    non_dominated = evolver.population.objectives[evolver.population.non_dominated_objectives(),:]
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(non_dominated)

    results = kmeans.cluster_centers_

    closest, _ = pairwise_distances_argmin_min(results, non_dominated)

    #print (closest)
    return closest






In [3]:
evolver_rvea = RVEA(problem, interact=True, n_iterations=5, n_gen_per_iter=300)

evolver_moead = IMOEA_D(problem, interact=True, n_iterations=5, n_gen_per_iter=300)

PCP(evolver_rvea, evolver_moead)
plot_RVs(evolver_rvea)
plot_RVs(evolver_moead)


[[1.47157764 1.19801105 0.27505746]
 [0.71312326 1.3267265  0.57655197]
 [0.28215336 0.39650287 1.68480662]
 [0.66372458 0.93888813 1.17660783]
 [0.30262353 0.53575377 1.75813211]
 [0.74229618 0.90553599 2.09687027]
 [0.65403583 0.61147581 2.03630869]
 [0.9618091  1.34051938 1.46583276]
 [0.9318576  1.21785544 0.81648758]
 [0.77387674 1.04443392 1.17064789]
 [0.30188354 0.23780044 2.00297655]
 [1.67843122 1.25305362 0.62722232]
 [0.77398574 1.41270076 0.1069652 ]
 [0.85853238 0.50423008 1.57363178]
 [0.46478171 0.39872472 1.59605643]
 [0.71393411 1.27737042 0.95238057]
 [0.99502459 0.8968142  1.1183286 ]
 [0.60425862 0.66669693 1.40542815]
 [0.44186918 0.43402204 1.49929245]
 [1.05383752 0.56355834 1.72845354]
 [1.36528984 0.79689072 0.67745776]
 [1.01125463 2.00082046 0.12819587]
 [0.2422597  0.15485211 2.44857338]
 [0.09359733 0.17513677 1.90640855]
 [1.31608627 1.18294116 0.83637562]
 [1.95374028 1.13384953 0.55807463]
 [0.80550891 0.98191561 0.89412409]
 [0.27700291 0.41171953 1.38

In [4]:
pref_rvea, plot_rvea = evolver_rvea.start()
pref_moead, plot_moead = evolver_moead.start()

In [5]:
print(plot_rvea.content["dimensions_data"])
print(plot_moead.content["dimensions_data"])

                  f1         f2          f3
minimize           1          1           1
ideal     0.00536512  0.0102098  0.00790629
nadir            inf        inf         inf
                  f1         f2          f3
minimize           1          1           1
ideal     0.00536512  0.0102098  0.00790629
nadir            inf        inf         inf


In [6]:
pref_rvea[2].response = pd.DataFrame([[0.1,0.1,0.1]],
                             columns=pref_rvea[2].content['dimensions_data'].columns)

pref_moead[2].response = pd.DataFrame([[0.1,0.1,0.1]],
                             columns=pref_moead[2].content['dimensions_data'].columns)

In [7]:
pref_rvea, plot_rvea = evolver_rvea.iterate(pref_rvea[2])

pref_moead, plot_moead = evolver_moead.iterate(pref_moead[2])

selected = cluster_select(evolver_moead, 5)

PCP(evolver_rvea, evolver_moead, selected)
plot_RVs(evolver_rvea)
plot_RVs(evolver_moead)

reference point
[[0.63149394 0.81183863 0.59475765]
 [0.55943792 0.59841568 0.64843018]
 [0.60787073 0.70392109 0.59229716]
 [0.56846929 0.68553966 0.70281807]
 [0.63665278 0.60745791 0.53684442]]


In [8]:
selected = cluster_select(evolver_rvea, 5)

PCP(evolver_moead, evolver_rvea, selected)

[[0.61669757 0.61744543 0.52678308]
 [0.61568517 0.83429259 0.67158644]
 [0.57922347 0.66464409 0.6507012 ]
 [0.63857496 0.84075965 0.62270585]
 [0.62612429 0.74543656 0.56404673]]


In [9]:
#pref_rvea[3].content['dimensions_data'].columns = 5
#preferred_range = np.array([[5.6,3.1,6.5,-3.0,0.2],[5.8,3.2,7.0,-2.0,0.3]])
#preferred_range  = np.array([[5.6,5.8],[3.1,3.2],[6.5,7.0],[-3.0,-2.0],[0.2,0.3]])
preferred_range  = np.array([[0.65,0.7],[0.65, 0.7],[0.6,0.65]])
pref_rvea[3].response = preferred_range
pref_moead[3].response = preferred_range

In [10]:
pref_rvea, plot_rvea = evolver_rvea.iterate(pref_rvea[3])
pref_moead, plot_moead = evolver_moead.iterate(pref_moead[3])


In [11]:
selected = cluster_select(evolver_rvea, 5)
PCP(evolver_moead,evolver_rvea, selected)
print(selected)

[[0.58619349 0.5960848  0.5681967 ]
 [0.59081409 0.58581354 0.56481747]
 [0.60309777 0.61243488 0.5344348 ]
 [0.59366402 0.59597128 0.55149204]
 [0.60159881 0.60216191 0.54973911]]


[24 17 30 15 16]


In [12]:
print(evolver_moead.population.objectives[20,:])
print(evolver_rvea.population.objectives[8,:])

[0.60630207 0.58972898 0.56644396]
[0.60621467 0.60783687 0.54129269]


In [13]:
selected_solution_rvea = np.array([8])
selected_solution_moead = np.array([20])

pref_rvea[0].response = np.array([8])
pref_moead[0].response = np.array([20])

In [14]:
pref, plot = evolver_rvea.iterate(pref_rvea[0])
pref, plot = evolver_moead.iterate(pref_moead[0])

selected = cluster_select(evolver_rvea, 5)
PCP(evolver_moead, evolver_rvea,selected)

[[0.60846924 0.58910547 0.57033377]
 [0.59081409 0.58581354 0.56481747]
 [0.60056043 0.59335948 0.55026263]
 [0.74220266 1.13460288 0.        ]
 [0.59274023 0.59810475 0.55069253]]
